In [28]:
import geopandas as gpd
import pandas as pd
import os


In [29]:
# === Пути к данным ===
base_path = "../data/processed"

stops_path = os.path.join(base_path, "transport_stops.geojson")
routes_path = os.path.join(base_path, "transport_routes.geojson")
sport_stops_path = os.path.join(base_path, "sport_with_stops.geojson")
output_path = os.path.join(base_path, "sport_with_routes.geojson")

In [30]:
# === Загрузка данных ===
stops = gpd.read_file(stops_path)
routes = gpd.read_file(routes_path)
sport_with_stops = gpd.read_file(sport_stops_path)

In [31]:
# === Приведение к одной проекции (метры) ===
stops = stops.to_crs("EPSG:3857")
routes = routes.to_crs("EPSG:3857")

In [32]:
# === Пространственное соединение: какие маршруты проходят через остановку ===
# Буфер вокруг остановки (например, 200 м) чтобы попасть в линию маршрута
stop_buffer = stops.copy()
stop_buffer["geometry"] = stop_buffer.geometry.buffer(200)

In [33]:
# Присваиваем маршруты остановкам
stops_with_routes = gpd.sjoin(routes, stop_buffer, how="inner", predicate="intersects")

# Выбираем и переименовываем нужные колонки
stops_with_routes = stops_with_routes[["global_id_right", "StationName", "RouteNumber", "TypeOfTransport"]]
stops_with_routes = stops_with_routes.rename(columns={
    "global_id_right": "stop_global_id",
    "StationName": "station_name",
    "RouteNumber": "route_number",
    "TypeOfTransport": "transport_type"
})


In [34]:
# === Объединение с таблицей спортобъектов ===
result = sport_with_stops.merge(stops_with_routes, on="stop_global_id", how="left")

In [35]:
# === Сохранение результата ===
os.makedirs(os.path.dirname(output_path), exist_ok=True)
result.to_file(output_path, driver="GeoJSON")

print(f"[INFO] Сопоставлено маршрутов: {result['route_number'].notnull().sum()}")
print(f"[INFO] Сохранено в: {output_path}")


[INFO] Сопоставлено маршрутов: 284
[INFO] Сохранено в: ../data/processed\sport_with_routes.geojson
